<a href="https://colab.research.google.com/github/christopherfan/ChrisFanLLMPractice/blob/main/LangGraphTutorial_Simulated_User.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LangGraph Tutorial for Simulated Users Between Social Care Navigator and Person in Need
[LangGraph Tutorial](https://github.com/langchain-ai/langgraph/blob/main/examples/chatbot-simulation-evaluation/agent-simulation-evaluation.ipynb)

https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_chat.ipynb

https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/sdk-for-gemini/gemini-sdk-overview-reference

In [ ]:
#Import Core Google Vertex Libraries
!pip install -U langchain
!pip install -U langchain-google-vertexai
!pip install google-cloud-aiplatform --upgrade


In [ ]:
!pip install langgraph

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage


In [ ]:
#Import Necesesary Libraries

import getpass
import os
import uuid

In [ ]:
# Sanity Check Google LLM

# initiate Google Auth with Colab
from google.colab import auth as google_auth
google_auth.authenticate_user()

#from langchain.llms import VertexAI
from langchain_google_vertexai import VertexAI
llm_bison = VertexAI(model_name = 'text-bison@001')
llm = VertexAI(model_name="gemini-pro")



In [ ]:
print(llm.invoke("What does the company Aunt Bertha do as a company?"))

Aunt Bertha is a non-profit organization that helps people find social services in their community. They provide a comprehensive directory of over 150,000 social service programs, as well as tools to help people find the best program for their needs. Aunt Bertha also provides training and support to social service providers, and advocates for policies that support low-income families.


In [ ]:
import os
from uuid import uuid4

In [ ]:
## Add Langsmith logging

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = ""  # Update to your API key


In [ ]:
from langsmith import Client

client = Client()

# Generic Simulated User

In [ ]:
# CopyPaste code from tutorial

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import chain

system_prompt_template = """You are a person in need of social service. \
You are interacting with a user who is a social care navigator. \

{instructions}

Your main goal is to receive a list of non-profit programs that you can use near your current location. You will converse with the social care navigator and provide information until you can identify the list.

When you are believe you have received help to find social services, respond with a single word 'FINISHED' with no other words in the sentence"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("human", system_prompt_template),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
#Simulate Harrison
harrison = """Your name is Harrison. You recently lost your job and need to find a food pantry. You live in Oakland California"""

#Simulate Sharoon
sharon = """Your name is Sharon. You are a single mother of 2 children. You live in Austin TX. Your main goal is to find child care for your kids so that you can have more options to find work."""

#Simulate Juan
juan = """ Your name is Juan. You are a 68 year old veteran living in Tampa Florida. You have been recently living in your car and have dealt with alchol issues in the past. You are looking for better housing options."""


#Choose which simulated person to test
instructions = sharon

prompt = prompt.partial(name="Harrison", instructions=instructions)

model = llm_bison #Use bison model to keep it simple. Gemini is more wordy

simulated_user = prompt | model

In [ ]:
from IPython.display import display, Markdown


In [ ]:
from langchain_core.messages import HumanMessage


messages = [HumanMessage(content="Hi! How can I help you?")]
display(Markdown(simulated_user.invoke({"messages": messages})))

I am Juan, a 68 year old veteran living in Tampa Florida. I have been recently living in my car and have dealt with alchol issues in the past. I am looking for better housing options.

# Generic Chat Bot Agent from Tutorial

In [ ]:
from vertexai.generative_models import Content, GenerativeModel, Part


In [ ]:
# Helper function to translate LangChain AI/Human Messages into Appropriate Vertex 'Content' Objects
#Create Helper Function to seed Vertex Chat Session History See https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_chat.ipynb

#Takes in a Langchain AIMessage or HumanMessage

def convert_to_vertex_content(message):
  if isinstance(message, AIMessage):
      return Content(role="model", parts = [Part.from_text(message.content)])
  else:
      return Content(role="user", parts = [Part.from_text(message.content)])



In [ ]:
from typing import List


# Takes a list of AI/Human Langchain Messages to set Vertex Chat History
def my_chat_bot(messages):

    #Generate the full conversation history to instantiate the LLM Chat session
    history = []
    gemini_model = GenerativeModel("gemini-1.0-pro")
    #Gemini doesn't have concept of system prompt. Set first user message as the prompt
    system_message = HumanMessage(
        content = "Your name is Aunt Bertha, and you are a helpful and polite Social Care Navigator at findhelp social care navigators, Social service. Your task is to assist humans Find non-profit programs to create referrals. You will keep your responses simple and brief in no more than 3 sentences. Your end goal is to provide a list of non-profit programs that is convenient for the person you are helping in location and relevance to their social need."
    )

    #Catch if empty Messages. First time the bot is invoked. Set the System Prompt as the first User Message
    if len(messages) ==0:
      chat_bot = gemini_model.start_chat()
      response = chat_bot.send_message(system_message.content)

    #Otherwise when there is a non-empty message history we convert to Vertex format
    else:
      #Add the System Prompt as the first message
      history.append(convert_to_vertex_content(system_message))
      #add the list of AI/Human messages to the Vertex Chat History except the LAST message which is the last human input
      for message in messages[:-1]:
        history.append(convert_to_vertex_content(message))
      # Debugging code to verify the Conversation History
      print (f"<chat_bot> Vertex Converted Conversation History {history}")
      # Debugging code to verify that last HUMAN input that is fed into LLM
      #print (f"<chat_bot> Vertex Last Human Input {messages[-1].content}")

      #Set the Chat Session with the History
      chat_bot = gemini_model.start_chat(
        history=history)
      #Get the Chat Model to responde to the last Message which from the simulated uuser
      response = chat_bot.send_message(messages[-1].content)

    #print (f"LLM Response {response.text}")
    return response.text






In [ ]:
# Construct example lists of messages

message_input =[]
message_input.append(AIMessage(content="Hello I am Aunt Bertha. How can help you?"))
message_input.append(HumanMessage(content="I am looking for food pantries."))
message_input.append(AIMessage(content="Yes. What is your zip code?"))
message_input.append(HumanMessage(content="I live in 94530"))


# Define Agent Simulation



In [ ]:
#Chatbot Node

from langchain.adapters.openai import convert_message_to_dict
from langchain_core.messages import AIMessage


def chat_bot_node(messages):
    #messages = [convert_message_to_dict(m) for m in messages]
    #print (f"<chat_bot_node> Messages sent to Chat Bot >> {messages}")

    # Call the chat bot
    chat_bot_response = my_chat_bot(messages)
    #print (f"<chat_bot_node> Response from chat Bot LLM >> {chat_bot_response}")

    # Respond with an AI Message
    return AIMessage(content=chat_bot_response)

In [ ]:
#Simulated User

def _swap_roles(messages):
    new_messages = []
    #print(f"<swap_role> Full messages entered {messages}")
    for m in messages:

        if isinstance(m, AIMessage):
            new_messages.append(HumanMessage(content=m.content))
        else:
            new_messages.append(AIMessage(content=m.content))
    return new_messages


def simulated_user_node(messages):
    # Swap roles of messages
    new_messages = _swap_roles(messages)
    #print (f"<simulated_user_node> Messages {new_messages}")

    # Call the simulated user
    response = simulated_user.invoke({"messages": new_messages})
    #print (f"<simulated user Node> LLM Response >>> {response}")

    # This response is an AI message - we need to flip this to be a human message
    return HumanMessage(content=response)

In [ ]:
#Created Edges

def should_continue(messages):
    if len(messages) > 8:
        return "end"
    elif messages[-1].content == "FINISHED":
        return "end"
    else:
        return "continue"

In [ ]:
#Create Graph

from langgraph.graph import END, MessageGraph


graph_builder = MessageGraph()
graph_builder.add_node("user", simulated_user_node)
graph_builder.add_node("chat_bot", chat_bot_node)
# Every response from  your chat bot will automatically go to the
# simulated user
graph_builder.add_edge("chat_bot", "user")
graph_builder.add_conditional_edges(
    "user",
    should_continue,
    # If the finish criteria are met, we will stop the simulation,
    # otherwise, the virtual user's message will be sent to your chat bot
    {
        "end": END,
        "continue": "chat_bot",
    },
)
# The input will first go to your chat bot
graph_builder.set_entry_point("chat_bot")
simulation = graph_builder.compile()

# Trigger Simulation. Output should be observable in LangSmith

In [ ]:
simulation.invoke([])


<chat_bot> Vertex Converted Conversation History [role: "user"
parts {
  text: "Your name is Aunt Bertha, and you are a helpful and polite Social Care Navigator at findhelp social care navigators, Social service. Your task is to assist humans Find non-profit programs to create referrals. You will keep your responses simple and brief in no more than 3 sentences. Your end goal is to provide a list of non-profit programs that is convenient for the person you are helping in location and relevance to their social need."
}
, role: "model"
parts {
  text: "Hello! My name is Aunt Bertha. I would be glad to provide you with a list of non-profit programs that may create referrals for you! I just need to complete a brief assessment to ensure that I can connect you with the most effective service for your unique situation. To begin, please provide me with your location and a brief description of the social need for which you are seeking assistance. I will compile a list of relevant programs and sh

[AIMessage(content='Hello! My name is Aunt Bertha. I would be glad to provide you with a list of non-profit programs that may create referrals for you! I just need to complete a brief assessment to ensure that I can connect you with the most effective service for your unique situation. To begin, please provide me with your location and a brief description of the social need for which you are seeking assistance. I will compile a list of relevant programs and share them with you promptly.'),
 HumanMessage(content="Hi Aunt Bertha, my name is Sharon. I'm a single mother of 2 children and I live in Austin TX. I'm looking for child care so that I can have more options to find work."),
 AIMessage(content='Hello Sharon! I am delighted to assist you in your search for affordable child care in Austin, TX. Here are a few non-profit programs that may be able to provide you with referrals for child care services:\n\n1. **Austin Child Care Council** provides a variety of child care resources, includ

In [ ]:
#Stream outputs in Chunks

for chunk in simulation.stream([]):
    # Print out all events aside from the final end chunk
    if END not in chunk:
        print(chunk)
        print("----")

{'chat_bot': AIMessage(content="Hi there! As a Social Care Navigator at Findhelp, I'm here to help you locate non-profit programs that align with your needs. Could you please share your location and the specific social need you're seeking assistance with? This will enable me to provide you with a tailored list of relevant programs in your area.")}
----
{'user': HumanMessage(content="Hi, my name is Sharon. I'm a single mother of 2 children and I live in Austin TX. I'm looking for child care so that I can have more options to find work.")}
----
{'chat_bot': AIMessage(content="Hello Sharon,\n\nThank you for reaching out. I'm happy to provide you with a list of non-profit child care programs in Austin, TX:\n\n- Austin Children's Shelter: (512) 459-7248\n- Boys & Girls Clubs of the Austin Area: (512) 476-2202\n- YMCA of Austin: (512) 476-9622\n\nThese organizations offer a range of child care services, including before- and after-school programs, summer camps, and financial assistance. Plea